<a href="https://colab.research.google.com/github/soumitkundu/SIG742-EndTermAssignment/blob/main/SIG742Task2-Group-13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Student ID:** _225738919_

**Student Name:** _Soumit Kundu_

**Workshop / Lab Session Time:** Mon / Tue / Wed / Thu / Fri

# Part 1: **Data Acquisition and Manipulation**

The (business_review_submission.zip) data used for this part could be found in [here](https://github.com/tulip-lab/sit742/blob/develop/Jupyter/data/business_review_submission.zip). There are two files in the data. The first one is about the business review submission with many companies. For each of the row, the review submission is provided with relevant information such as user id, time, name and many others. The second one is the meta information of the business and the two data could be joined with gmap_id. You will need to use spark to **first read the unzipped (csv) review data for starting** and later join the meta review business data on dataframe (pandas or spark). You could find the code on reading csv data with Spark from [M04G](https://github.com/tulip-lab/sit742/blob/develop/Jupyter/M04-DataManipulation/M04G-SparkSQL.ipynb). In some of the tasks, if the question is not specifically asking to use spark, you could use both pandas and numpy.

### Answer 1.1

**Question:**

Using PySpark to do some data wrangling process, so that:
> **1.1.1** For the none or null in text column, change it to 'no review'.

> **1.1.2** Process the content in time column, and convert the strings from time to yyyy-mm-dd format in the new column as newtime and show the first 5 rows.

**Installing Required Libraries**

In [1]:
#update local version of the package catalog
!apt-get update
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.3.3
# !wget -q https://archive.apache.org/dist/spark/spark-3.3.3/spark-3.3.3-bin-hadoop3.tgz
# # unzip it
# !tar xf spark-3.3.3-bin-hadoop3.tgz
# install findspark
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://cli.github.com/packages stable InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,690 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,577 kB]
Ge

In [2]:
import os

import findspark
findspark.init()

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [5]:
import pyspark
pyspark.__version__

'3.5.1'

In [6]:
# from pyspark import SparkContext
# from pyspark.sql import SQLContext

# # Initialize SparkContext
# sc = SparkContext.getOrCreate()

# #SQLContext is the entry point for working with structured data (rows and columns) in Spark, in Spark 1.x.
# #As of Spark 2.0, this is replaced by SparkSession.
# #However, we are keeping the class here for backward compatibility.
# #A SQLContext can be used create DataFrame, register DataFrame as tables, execute SQL over tables, cache tables, and read parquet files
# sqlContext = SQLContext(sc)

In [4]:
# Importing SparkSession as our pyspark version is updated
from pyspark.sql import SparkSession

# Initializing SparkSession and crearing the app
spark = SparkSession.builder.appName('SIG742').getOrCreate()
spark

In [5]:
# connect to google colab drive
from google.colab import drive
drive.mount("/content/gdrive")

# Initializing folder path where our datasets are stored
drive_path = '/content/gdrive/MyDrive/MDS-Datasets/SIG742/business_review_submission'

Mounted at /content/gdrive


In [ ]:
#Returns a DataFrameReader that can be used to read data in as a DataFrame.
# df_review = sqlContext.read.csv(drive_path + "/review.csv", header=True)
# df_review

DataFrame[user_id: string, name: string, time: string, rating: string, text: string, pics: string, resp: string, gmap_id: string]


In [6]:
df_review = spark.read.csv(drive_path + "/review.csv", header=True)
df_review

DataFrame[user_id: string, name: string, time: string, rating: string, text: string, pics: string, resp: string, gmap_id: string]

In [10]:
df_review.show(5)

+--------------------+--------------------+-------------+--------------------+--------------------+----+----+--------------------+
|             user_id|                name|         time|              rating|                text|pics|resp|             gmap_id|
+--------------------+--------------------+-------------+--------------------+--------------------+----+----+--------------------+
|1.091298048426862...|          Nicki Gore|1566331951619|                   5|We always stay he...|NULL|NULL|0x56b646ed2220b77...|
|1.132409264057589...|       Allen Ratliff|1504917982385|                   5|Great campground ...|NULL|NULL|0x56b646ed2220b77...|
|1.130448378911412...|   Jonathan Tringali|1474765901185|                   4|We tent camped he...|NULL|NULL|                NULL|
|There is a bath h...| 2 restrooms (sin...|       toilet| shower). The hot...| but they lack ve...|NULL|NULL|                NULL|
|Wi-Fi didn't reac...|                NULL|         NULL|0x56b646ed2220b77...|     

In [7]:
# Importing spark sql functions
from pyspark.sql.functions import *

**1.1.1** For the none or null in text column, change it to 'no review'.

In [8]:
null_review_count = df_review.filter(col('text').isNull()).count()
print(f"None or Null count for text column is {null_review_count}")

None or Null count for text column is 249931


In [9]:
df_review_filtered = df_review.fillna('no review', subset=['text'])

In [10]:
no_review_count = df_review_filtered.filter(df_review_filtered['text'] == 'no review').count()
no_review_count

249931

**1.1.2** Process the content in time column, and convert the strings from time to yyyy-mm-dd format in the new column as newtime and show the first 5 rows.

In [11]:
df_converted = df_review_filtered.withColumn(
    'newtime',
    to_date(from_unixtime(col('time').cast('bigint') / 1000))
)
df_converted.show(5)

+--------------------+--------------------+-------------+--------------------+--------------------+----+----+--------------------+----------+
|             user_id|                name|         time|              rating|                text|pics|resp|             gmap_id|   newtime|
+--------------------+--------------------+-------------+--------------------+--------------------+----+----+--------------------+----------+
|1.091298048426862...|          Nicki Gore|1566331951619|                   5|We always stay he...|NULL|NULL|0x56b646ed2220b77...|2019-08-20|
|1.132409264057589...|       Allen Ratliff|1504917982385|                   5|Great campground ...|NULL|NULL|0x56b646ed2220b77...|2017-09-09|
|1.130448378911412...|   Jonathan Tringali|1474765901185|                   4|We tent camped he...|NULL|NULL|                NULL|2016-09-25|
|There is a bath h...| 2 restrooms (sin...|       toilet| shower). The hot...| but they lack ve...|NULL|NULL|                NULL|      NULL|
|Wi-Fi

### Answer 1.2

**Question:**

Find out the information for gmap_id on the reviews. In order to achieve the above, some wrangling work is required to be done:
> **1.2.1** Using pyspark to calculate the number of reviews per each unique gmap_id and save as float format in pyspark dataframe to show the top 5 rows.

> **1.2.2** Transform the current pyspark dataframe to pandas dataframe (named as df) and create the column reivew_time with the information of review time on hours level. Print your df pandas dataframe with top 5 rows after creating the column review_time.

> **1.2.3** Using matplotlib or seaborn to draw some (two or more if possible) visualizations on the relationship between gmap_id and reivew_time. You could explore for example, what is the time people usually review? How many business is reviewed in the morning time etc. Please also discuss the insights you are finding with your visualizations in the markdown cell. Please also include your findings and visualizations in the report.

### Answer 1.3

**Question:**

Let’s continue to analyze the reivew_time with reviews and related gmap_id. You need to use another data meta-business to join with the current dataframe on gmap_id.
> **1.3.1** Determine which workday (day of the week), generates the most reviews (plotting the results in a line chart with workday on averaged submissions).

> **1.3.2** Identify the names of business (column name from data meta-business) that has the highest averaged ratings on ‘that workday’ (you need to find out from 1.3.1), and find out which category those businesses are from?

> **1.3.3** Please further explore the data on name of business and find out some more insights by yourself such as which category it is and what are the peak hours etc. Please use visualizations and tables to support your findings and write down the insights in the markdown cell. Please also include your findings and visualizations in the report.

### Answer 1.4

**Question:**

For the reviews on each of the submissions, work on all the review content and find out the top 30 most common words; Also generate separate word cloud visualizations for different years by grouping the reviews by review year and write down the insights in the markdown cell. Please also include your findings and visualizations in the report.

### Answer 1.5

**Question:**

Let’s do some analysis on the business_name and the reviewers. Determine the number of unique reviewers of business and its categories to identify which business / category has attracted the most reviewers (find out the highest distinct count of reviewers on business / category level). Also, analyze the temporal patterns of when reviewers submitted their reviews (you could leverage the workday, year, month, or hours to conduct the analysis) and share your findings and insights in the markdown cell. Please also include your findings and insights (visualizations) in the report.

### Answer 1.6

**Question:**

As the data scientist, you are required to build a recommendation for the business by using reviews, ratings, and its categories. In this task, you need to:
> **1.6.1** Write down your strategy of building the recommendation on business for customers in the markdown cell. You could create your own strategy or leverage the provided one here KNN on collaborative filtering. Please also include your strategy details in the report.

> **1.6.2** Could you please try to implement the strategy (code) you have written down for the recommendation system? Please give detailed explanation of your code and the logic in the comments and also interpret the recommendations with examples in the markdown cell. Please also include your implementation details and results in the report.

### Answer 1.7

**Question:**

Continue work on the review data you have now, for each of the submissions of the review, you will need to explore the rating with other information:
> **1.7.1** Build visualization to explore the relationships of the rating and business categories. Please write down your insights in the markdown cell and also include your insights and visualizations in the report.

> **1.7.2** Let’s focus on the lower ratings now. Could you please find out the actual reviews on lower ratings and analyze on the reason? (You could use the common used words in lower rating reviews or design your own strategy with reasonable logic). Please also include your analysis details in the report.

### Answer 1.8

**Question:**

Continue to work on the submission of the reviews, we would like to focus on the reviewer level with all the reviewed business history, but before we actually conduct the programming, we will need to finish few questions for exploration:
> **1.8.1** Check on the reviewer level reviewed business, sort the review of each business by the review time (newtime column) and then save the business name into the list variable user_business_list for each reviewer.

> **1.8.2** Check on the user_business_list, could you observe some repeated business names for the same user? If so, could you remove those duplicated business names under same user? Please print out the number of element in the user_business_list for each reviewer before removing the duplicated business name and after removing the duplicated business name.

> **1.8.3** Check on the user_business_list, could you find the user similarities according to their past reviewed business ? You are free to design your own strategy and give sufficient explanation in markdown cell and code implementation together. Please also include your strategy details and implementation in the report.

_Hint: you might consider to use encoding for each of the business names and then calculate the difference of the users._

# Part II: **Submission Prediction**

### Answer 2.1

**Question:**

In this question, we will focus only on two information: total reviews per day with review time (newtime from the dataframe) to form the review volume time series. You are required to explore the review time series. There are some days not available in the review time series. Please add those days into the review time series with default number of review with the mean value of the number of review per day in the whole data (without any filtering on reviews). After that, decompose the submission review time series with addictive mode and analyses on the results to find if there is any seasonality pattern (you could leverage the M05A material from lab session with default setting in seasonal_decompose function). Please also include your analysis details and implementation in the report.

### Answer 2.2

**Question:**

We will try to use time series model ARIMA for forecasting the future. You need to find the best model with different parameters on ARIMA model. The parameter range for p,d,q are all from [0, 1, 2]. In total, you need to find out the best model with lowest Mean Absolute Error from 27 choices (you might need to split the time series to train and test with yourself with grid search according to the M05B material). Also, you are required to discuss with your group member on exploring the deep learning time series forecasting methods such as LSTM and RNN. Please write down your discussion around the necessary data wrangling and modeling steps (steps on how to achieve, not actual code). Also please give the reference of the deep learning time series forecasting models you are using. Please also include your discussion details and implementation in the report.

**Question:**

In this question, you are provided with the PDF file by Universities Australia via Indigenous Strategy annual report. You are required to critically analyze this report using your data science skills.
> **Data Extraction** Carefully review the PDF and identify all relevant quantitative data, tables, and figures that can be extracted or digitized; Present any extracted data in a structured format (e.g., CSV, Excel table, or DataFrame);

> **Data Analysis** Utilize your data analytics skills to discover common patterns or trends from the report; Where possible, compare trends over multiple years, between institutions, or across different Indigenous strategy metrics.

> **Insights** Provide a clear and concise summary of the main patterns, trends, or correlations discovered from your analysis; Interpret what these findings reveal about the progress and challenges of Indigenous strategies in Australian universities.

_You may use any data analytics tools or libraries you are comfortable with. All steps, from extraction to insights, should be clearly documented in your SIT742Task2Report.pdf, and source code should be in SIT742Task2Code.ipynb._

# Part III: **Optional: Questionnaire on Integrating Indigenous Perspectives into DS Education**

**Note:** This part of the assignment is **optional** and carries **no mark**.

In modern data-driven environments, collaboration often involves individuals from diverse cultural and social backgrounds. As part of our ongoing efforts to develop inclusive and socially responsible unit, we are exploring ways to meaningfully integrate Australian Indigenous perspectives into Data Science education, specifically in the context of the unit SIT742: Modern Data Science. We invite you to share your thoughts and feedback on this important topic. Your input will help us improve the quality and inclusiveness of both the unit and its associated learning materials.
If you are willing to assist, please consider completing a short questionnaire titled Integrating Indigenous Perspectives into Data Science Education, which can be accessed at the following link:
- [POST-Survey Questionnaire](https://researchsurveys.deakin.edu.au/jfe/form/SV_3dZpMmkbdjsKkaq)

(Please interpret its references to “ICT” as also including Data Science.)

#### Related Information
**Human Research Ethics Application ID:** 2025/HE000518 Approval Date: 28/04/2025 Estimated Time to Complete: Approximately 10 minutes

**Confidentiality:** All responses are strictly confidential and will be used solely for research and educational improvement purposes.

**University Policy** Deakin Indigenous Strategy 2023-2028